In [1]:
#pip install yfinance

In [2]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error #to calculate metrics for the model later
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
import ipywidgets as widgets
from ipywidgets import Layout

In [3]:
unique_stonk = np.array(["GOOG", "IBM", "MMM", "AMZN", "AAPL", "TSLA", "NIO", "NIU", "FSLR", "FB", "QRVO"])
stonk = widgets.SelectMultiple(
    options = unique_stonk.tolist(),
    value = ['GOOG'],
    description='Stonk',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
)

In [4]:
def rsi_cal(dta3):
    dta3['change_in_price'] = dta3['Close'].diff() #create the change in price column which calculates gains/losses

    #now we need to create an "up" and a "down" column, containing gains/losses, to calculate the Relative Strength
    up = dta3[['change_in_price']].copy()
    down = dta3[['change_in_price']].copy()

    #up only contains gains, losses are marked as 0. down only contains losses, gains are marked as 0
    for i in range(len(up['change_in_price'])):
        if up['change_in_price'][i] < 0: 
            up['change_in_price'][i] = 0
        else:
            pass

    for i in range(len(down['change_in_price'])):
        if down['change_in_price'][i] >0: 
            down['change_in_price'][i] = 0
        else:
            down['change_in_price'][i] = abs(down['change_in_price'][i]) #we want all changes to be in absolute value


    #Avoid data leakage
    df_up = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_up = df_up.append(up['change_in_price'], ignore_index=True)
    df_up.drop(df_up.tail(1).index,inplace=True)
    up['change_in_price'] = df_up
    df_down = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_down = df_down.append(down['change_in_price'], ignore_index=True)
    df_down.drop(df_down.tail(1).index,inplace=True)
    down['change_in_price'] = df_down


    # Calculate the EWMA (Exponential Weighted Moving Average)
    ewma_up = up['change_in_price'].transform(lambda x: x.ewm(span = 14).mean())
    ewma_down = down['change_in_price'].transform(lambda x: x.ewm(span = 14).mean())

    # Calculate the RS
    RS = ewma_up/ewma_down

    # Calculate the RSI
    RSI = 100.0 - (100.0 / (1.0 + RS))

    dta3['down_days'] = down['change_in_price']
    dta3['up_days'] = up['change_in_price']
    dta3['RSI'] = RSI
    #We will drop the Nas latter
    return dta3

In [5]:
def stoc_oscillator_cal(dta3):
    #Creating the high and low columns
    high14 = dta3[['High']].copy()
    low14 = dta3[['Low']].copy()
    close_pushed = dta3['Close'].copy()
    df_high = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_high = df_high.append(high14, ignore_index=True)
    df_high.drop(df_high.tail(1).index,inplace=True)
    high14 = df_high
    df_low = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_low = df_low.append(low14, ignore_index=True)
    df_low.drop(df_low.tail(1).index,inplace=True)
    low14 = df_low
    df_close = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_close = df_close.append(close_pushed, ignore_index=True)
    df_close.drop(df_close.tail(1).index,inplace=True)
    close_pushed = df_close


    # We use the roll function to calculate the H14 and L14
    high14 = high14['High'].transform(lambda x: x.rolling(window = 14).min())
    low14 = low14['Low'].transform(lambda x: x.rolling(window = 14).max())

    # Calculation of Stochastic Oscillator.
    stoc_oscillator = 100 * ((close_pushed - low14) / (high14 - low14))
    dta3['low14'] = low14
    dta3['high14'] = high14
    dta3['k_percent'] = stoc_oscillator
    return dta3

In [6]:
def prc_cal(dta3):
    #the pct_change function helps us calculate the percentage change compared to n periods before
    close_pushed = dta3['Close'].copy()
    df_close = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_close = df_close.append(close_pushed, ignore_index=True)
    df_close.drop(df_close.tail(1).index,inplace=True)
    close_pushed = df_close
    dta3['PRC'] = close_pushed.transform(lambda x: x.pct_change(periods = 50))
    return dta3

In [7]:
def macd_cal(dta3):
    #Again, push to avoid data leakage
    close_pushed = dta3['Close'].copy()
    df_close = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_close = df_close.append(close_pushed, ignore_index=True)
    df_close.drop(df_close.tail(1).index,inplace=True)
    close_pushed = df_close

    #Again, use the ewm function like above for calculating EMA of the last 26 and 12 days
    ema26 = dta3['Close'].transform(lambda x: x.ewm(span = 26).mean())
    ema12 = dta3['Close'].transform(lambda x: x.ewm(span = 12).mean())
    macd = ema12 - ema26
    #Signal line
    ema_9_macd = macd.ewm(span = 9).mean()
    dta3['MACD'] = macd
    dta3['MACD_EMA'] = ema_9_macd
    return dta3

In [8]:
def r_william_cal(dta3):
    #Calculation process is very similar to the Stochastic Oscillator. Again, we push to avoid data leakage
    high14 = dta3[['High']].copy()
    low14 = dta3[['Low']].copy()
    close_pushed = dta3['Close'].copy()
    df_high = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_high = df_high.append(high14, ignore_index=True)
    df_high.drop(df_high.tail(1).index,inplace=True)
    high14 = df_high
    df_low = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_low = df_low.append(low14, ignore_index=True)
    df_low.drop(df_low.tail(1).index,inplace=True)
    low14 = df_low
    df_close = pd.Series([np.nan]) #add one more NaN value at the beginning
    df_close = df_close.append(close_pushed, ignore_index=True)
    df_close.drop(df_close.tail(1).index,inplace=True)
    close_pushed = df_close

    # We use the roll function to calculate the H14 and L14
    high14 = high14['High'].transform(lambda x: x.rolling(window = 14).min())
    low14 = low14['Low'].transform(lambda x: x.rolling(window = 14).max())

    # Calculate William %R 
    r_william = ((high14 - close_pushed) / (high14 - low14)) * - 100
    dta3['r_william'] = r_william
    return dta3


In [9]:
def randomforest(stonk):
    results={}
    for ticker in stonk:
        stock = yf.Ticker(ticker)
        # get historical market data
        dta = stock.history(period="759d") #5 years - each year has 253 trading days
        dta['Date']=dta.index
        dta.reset_index(drop=True, inplace=True)
        #Start adding the features
        dta=rsi_cal(dta)
        dta=stoc_oscillator_cal(dta)
        dta=prc_cal(dta)
        dta=macd_cal(dta)
        dta=r_william_cal(dta)
        dta=dta.dropna()
        #Training the model
        total = 759
        valid = int(total*0.4) # 40% of the data like above as test data
        train = dta.count()[0] - valid #60% of the data to train

        #make a copy so we leave the dta3 data set unchanged
        dta_copy = dta
        #including different features from above
        features = ['Open','Close', 'Low', 'High', 'Volume','RSI','k_percent', 'PRC', 'r_william', 'MACD']
        x_dta = dta_copy[features]
        y_dta = dta_copy['Close']
        x_training = x_dta[:train]
        x_val = x_dta[train:]
        y_training = y_dta[:train]
        y_val = y_dta[train:]
        x_val.drop(['RSI','k_percent', 'PRC', 'r_william', 'MACD'], axis=1)
        #this makes the test data to be completely independent of the training data - I removed all the technical features

        model = RandomForestRegressor(bootstrap=True)
        model.fit(x_training, y_training)
        forecast_forest = model.predict(x_val)
        real = y_dta[train:]

        # report performance
        mse = mean_squared_error(real, forecast_forest)
        rmse = math.sqrt(mean_squared_error(real, forecast_forest))
        mape = np.mean(np.abs(np.array(forecast_forest) - real)/np.abs(real))
        forest_plot_data = y_training.tolist() + forecast_forest.tolist()
        plt.plot(forest_plot_data)
    results[ticker]=forecast_forest
    return results

In [10]:
widgets.interactive(randomforest, stonk=stonk)

interactive(children=(SelectMultiple(description='Stonk', index=(0,), layout=Layout(display='flex', height='80…

In [11]:
def randomforest_nographs(stonk):
    results={}
    for ticker in stonk:
        stock = yf.Ticker(ticker)
        # get historical market data
        dta = stock.history(period="759d") #5 years - each year has 253 trading days
        dta['Date']=dta.index
        dta.reset_index(drop=True, inplace=True)
        #Start adding the features
        dta=rsi_cal(dta)
        dta=stoc_oscillator_cal(dta)
        dta=prc_cal(dta)
        dta=macd_cal(dta)
        dta=r_william_cal(dta)
        dta=dta.dropna()
        #Training the model
        total = 759
        valid = int(total*0.4) # 40% of the data like above as test data
        train = dta.count()[0] - valid #60% of the data to train

        #make a copy so we leave the dta3 data set unchanged
        dta_copy = dta
        #including different features from above
        features = ['Open','Close', 'Low', 'High', 'Volume','RSI','k_percent', 'PRC', 'r_william', 'MACD']
        x_dta = dta_copy[features]
        y_dta = dta_copy['Close']
        x_training = x_dta[:train]
        x_val = x_dta[train:]
        y_training = y_dta[:train]
        y_val = y_dta[train:]
        x_val.drop(['RSI','k_percent', 'PRC', 'r_william', 'MACD'], axis=1)
        #this makes the test data to be completely independent of the training data - I removed all the technical features

        model = RandomForestRegressor(bootstrap=True)
        model.fit(x_training, y_training)
        forecast_forest = model.predict(x_val)
        real = y_dta[train:]

        # report performance
        mse = mean_squared_error(real, forecast_forest)
        rmse = math.sqrt(mean_squared_error(real, forecast_forest))
        mape = np.mean(np.abs(np.array(forecast_forest) - real)/np.abs(real))
        results[ticker]=forecast_forest.tolist()
    return results

In [12]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
np.random.seed(777)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [13]:
def annualise(allocation, mean_returns, covariance):
    """Function to annualise the standard deviation and return of each portfolio

    Parameters
    ----------
    allocation
        The weight given to each stock in the portfolio
    mean_returns
        The mean return of each stock in the portfolio
    covariance
        The covariance matrix of the stocks in the portfolio
    
    Returns
    -------
    std_annual
        annualised standard deviation
    return_annual
        annualised return

    """
    std_annual = np.sqrt(np.dot(allocation.T, np.dot(covariance, allocation))) * np.sqrt(252)
    return_annual = np.sum(mean_returns*allocation ) *252 #252 trading days per year
    return std_annual, return_annual

In [14]:
def create_random_portfolios(df, mean_returns, covariance, risk_free_rate):
    """Function to create 50000 random portfolios using the above stocks for the efficient frontier.

    Parameters
    ----------
    mean_returns
        The mean return of each stock in the portfolio
    covariance
        The covariance matrix of the stocks in the portfolio
    risk_free_rate
        Current risk-free rate. We want to invest the cash portion of the portfolio in risk-free treasury
    
    Returns
    -------
    results
        list of standard deviation, return and Sharpe ratio of each portfolio
    list_weights
        list of portfolios

    """
    list_weights = []
    results = np.zeros((3,1000)) #30000 portfolios
    for i in range(1000):
        allocation = np.random.random(len(df.columns)) #randoming the allocation 
        allocation = allocation/sum(allocation) #normalizing the randomed vectors so they sum to 1
        list_weights.append(allocation)
        std_dev = annualise(allocation, mean_returns, covariance)[0] #annualise the std and return of the portfolio
        returns = annualise(allocation, mean_returns, covariance)[1]
        results[0,i] = std_dev  
        results[1,i] = returns 
        results[2,i] = (returns - risk_free_rate) / std_dev #Sharpe ratio
    return results, list_weights

In [15]:
'''
Now we define function to plot the efficient frontier with the optimal portfolio highlighted using the results above.
The optimal portfolio is found by taking the portfolio with the maximum Sharpe ratio (excess returns given the risk)
among the 50000 portfolios that we randomly created. We also display the portfolio with the smallest possible 
risk.
'''

def display_simulated_ef_with_random(stonk):
    """Function graph the efficient frontier and print out the optimal portfolios

    Parameters
    ----------
    mean_returns
        The mean return of each stock in the portfolio
    covariance
        The covariance matrix of the stocks in the portfolio
    risk_free_rate
        Current risk-free rate. We want to invest the cash portion of the portfolio in risk-free treasury
    """
    price_dict=randomforest_nographs(stonk)
    df = pd.DataFrame(data=price_dict)
    returns = df.pct_change()
    mean_returns = returns.mean()
    covariance = returns.cov()
    risk_free_rate = 0.0083
    
    
    results = create_random_portfolios(df, mean_returns, covariance, risk_free_rate)[0]
    weights = create_random_portfolios(df, mean_returns, covariance, risk_free_rate)[1]
    #For the portfolio with maximum sharpe
    index_max_sharpe = np.argmax(results[2]) #the maximum sharpe ratio among 30000 portfolios randomed
    std_max_sharpe = results[0,index_max_sharpe]
    return_max_sharpe = results[1,index_max_sharpe]
    allocation_max_sharpe = weights[index_max_sharpe]
    #For the portfolio with the lowest risk
    index_min_risk = np.argmin(results[0])
    std_min_risk = results[0, index_min_risk]
    return_min_risk = results[1, index_min_risk]
    allocation_min_risk = weights[index_min_risk]
    #Printing out the results    
    print("Maximum Sharpe Ratio Portfolio Allocation\n")
    print("Annualised Return:", return_max_sharpe)
    print("Annualised Volatility:", std_max_sharpe)
    print(df.columns.tolist())
    print(allocation_max_sharpe)
    print("Minimum Volatility Portfolio Allocation\n")
    print("Annualised Return:", return_min_risk)
    print("Annualised Volatility:", std_min_risk)
    print(df.columns.tolist())
    print(allocation_min_risk)
    #Visualizing with a scatterplot
    plt.figure(figsize = (15,10))
    std_applicable = []
    return_applicable=[]
    for i in range(len(results[0,:])): #only display the upper part of the frontier
        if results[1,i] > return_min_risk:
            std_applicable.append(results[0,i])
            return_applicable.append(results[1,i])
        else:
            pass
    plt.scatter(std_applicable,return_applicable, marker='o', s=10) #all portfolios
    plt.scatter(std_max_sharpe,return_max_sharpe,marker='*',color='red',s=500, label='Maximum Sharpe ratio')
    plt.scatter(std_min_risk,return_min_risk,marker='*',color='yellow',s=500, label='Minimum volatility')
    plt.legend()
    plt.title('Efficient frontier with optimized portfolios')
    plt.xlabel('Risk')
    plt.ylabel('Return')
    plt.show()

In [16]:
widgets.interactive(display_simulated_ef_with_random, stonk=stonk)

interactive(children=(SelectMultiple(description='Stonk', index=(0,), layout=Layout(display='flex', height='80…